Exploratory Data Analysis - Jupyter Notebook
================================================

**Table of contents**<a id='toc0_'></a>    
- [Loading the Data](#toc1_)    
- [Data Overview](#toc2_)    
- [Date Conversion](#toc3_)    
- [Data Cleaning](#toc4_)    
  - [Detect missing values](#toc4_1_)    
  - [Redundancy](#toc4_2_)    
    - [Drop Columns](#toc4_2_1_)    
    - [Duplicates](#toc4_2_2_)    
- [Descriptive Statistics](#toc5_)    
- [Data Analysis](#toc6_)    
  - [Correlation](#toc6_1_)    
  - [Visualizing Time Series](#toc6_2_)    
  - [Decomposition of variables](#toc6_3_)    
  - [Distributions](#toc6_4_)    
  - [Get the best-fitted distribution](#toc6_5_)    
    - [n_sick](#toc6_5_1_)    
    - [calls](#toc6_5_2_)    
  - [Boxplot](#toc6_6_)    
  - [Skewness](#toc6_7_)    
  - [Stationarity](#toc6_8_)    
- [Data Quality](#toc7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

This jupyter notebook contains the results of the Exploratory Data Analysis (EDA).

Ideas are partly taken from the books:  
- Mukhiya, S. K., & Ahmed, U. (2020). Hands-On Exploratory Data Analysis with Python: Perform EDA Techniques to Understand, Summarize, and Investigate Your Data. Packt Publishing.
- Atwan, T. A. (2022). Time Series Analysis with Python Cookbook: Practical recipes for exploratory data analysis, data preparation, forecasting, and model evaluation. Packt Publishing.


# <a id='toc1_'></a>[Loading the Data](#toc0_)

Loading the data into a pandas DataFrame because it is widely used in data science (Mukhiya & Ahmed, 2020, p. 28).

In [ ]:
# Operating system functionalities to join pathname components
import os
# Pandas for DataFrame and CSV handling
import pandas as pd

# Join the filepath of the raw data file
filepath = os.path.join("..", "..", "..", "data", "raw", "sickness_table.csv")
print("### Load Data ### Filepath: ", filepath)

# Read the data from CSV file
data_raw = pd.read_csv(filepath)

# <a id='toc2_'></a>[Data Overview](#toc0_)

At first, get an overview about the data structure (Mukhiya & Ahmed, 2020, p. 30).

In [ ]:
# Print the first 5 rows
print("### Data Overview ### Structure (first 5 rows): \n", data_raw.head(5))

In [ ]:
# Print some information about the DataFrame
print("### Data Overview ### Data info: \n")
data_raw.info()

It is therefore multivariate time series data, with the following columns:
- (unnamed): Row number
- date: Date
- n_sick: Number of emergency drivers who have registered a sick call
- calls: Number of emergency calls
- n_duty: Number of emergency drivers on duty
- n_sby: Number of available substitute drivers
- sby_need: Number of substitute drivers to be activated
- dafted: Number of additional duty drivers that have to be activated if the number of on-call drivers are not sufficient

The columns contain either integer or float values with exception of the date.

# <a id='toc3_'></a>[Date Conversion](#toc0_)

Sometimes it makes sense to convert some fields to simplify further processing. In the present data set, this is the case for the date.

The date field is an object, so we need to convert it into a DateTime argument (Mukhiya & Ahmed, 2020, p. 76).

In [ ]:
# Copy the data for conversion into a new variable
data = data_raw.copy()
# Convert the date objects into DateTime (raise an exception when parsing is invalid)
data.date = data.date.apply(lambda x: pd.to_datetime(x, errors='raise', utc=True))
print("### Date Conversion ### First 5 rows after convert date: \n", data.head(5))
# Get the data types of the columns again
print("### Date Conversion ### Data types after convert date: \n",data.dtypes)

# <a id='toc4_'></a>[Data Cleaning](#toc0_)

Data cleansing is useful to avoid errors and misinterpretation of data.

## <a id='toc4_1_'></a>[Detect missing values](#toc0_)

Identify NaN Values within the pandas dataframe using the isnull() function (Mukhiya & Ahmed, 2020, p. 113).

In [ ]:
# Detect missing values: In each columns
print("### Data Cleaning ### Number of missing values in each column: \n", data.isnull().sum())
# Detect missing values: Total number
print("### Data Cleaning ### Total number of missing values: ", data.isnull().sum().sum())

There are no missing values within the dataset.

## <a id='toc4_2_'></a>[Redundancy](#toc0_)

### <a id='toc4_2_1_'></a>[Drop Columns](#toc0_)

Irrelevant columns can be dropped (Mukhiya & Ahmed, 2020, p. 79).

In [ ]:
# Discover columns that contain only a few different values
print("### Data Cleaning ### Number of particular values in each column: \n", data.nunique())

The first column only contains the row number, which is obsolete due to the row indices of the DataFrame and can be removed.  
The column n_sby contains only one particular value, which can be removed in general. However, this is the value to be predicted, so the column should remain to keep the data structure compatible for the future.

In [ ]:
# Drop the column with index 0 (unamed)
data.drop(columns=data.columns[0], inplace=True)
print("### Data Cleaning ### Raw data columns: \n",data_raw.columns)
print("### Data Cleaning ### Cleaned columns: \n",data.columns)


The column n_duty contains 3 different values. In what frequency do they occur?

In [ ]:
# Get the set of values witin the column and print the length
n_duty_set = set(data.n_duty)
print("### Data Cleaning ### Set of n_duty: \n",n_duty_set)
n_duty_set_len = len(n_duty_set)
print("### Data Cleaning ### Length of the set of n_duty: \n",n_duty_set_len)

# Plot the distribution
# Matplotlib for plots
import matplotlib.pyplot as plt
plt.figure().set_figwidth(15)
# Create a figure
fig, ax = plt.subplots()
# Set figure width [inch]
fig.set_figwidth(16)
# Plot data
ax.plot(data.date, data.n_duty, '.')
# Set x-label
ax.set_xlabel("date")
# Set y-label
ax.set_ylabel("n_duty")
# Set title
ax.set_title("Number of emergency drivers on duty")


The finding is, that the number of emergency drivers on duty was increased two times, the column can thus not be removed.

The column dafted contains the number of additional duty drivers that have to be activated if the number of on-call drivers are not sufficient. It can be assumed that this is the difference between sby_need and n_sby if it is positive. Let's check:

In [ ]:
# Get the rows if the value of the column dafted ist not zero:
data_dafted_rows = data.loc[(data['dafted'] != 0)]
print("### Data Cleaning ### Data rows with values dafted != 0: \n", data_dafted_rows)
# Counter for rows for which the assumption is false (the difference between sby_need and n_sby is equal to dafted in case dafted is not zero)
couter_dafted_is_required = 0
# Counter to check the number of loop passes
counter_loop_passes = 0
for index, row in data_dafted_rows.iterrows():
    counter_loop_passes += 1
    if ((row.sby_need - row.n_sby) != row.dafted):
        print("### Data Cleaning ### Check dafted: #### FALSE ####\n")
        # Increment the counter
        couter_dafted_is_required += 1
print("### Data Cleaning ### Check loop passes: ", counter_loop_passes)
print("### Data Cleaning ### Number of loops which have shown that the dafted column is required: ", couter_dafted_is_required)  

In this case, the column dafted could be dropped because it has a strong correlation with sby_need. But we leave the column in for now and analyze the correlation to confirm this finding on a later stage.

In [ ]:
# Drop the column dafted
#data.drop(columns='dafted', inplace=True)
print("### Data Cleaning ### Raw data columns: \n",data_raw.columns)
print("### Data Cleaning ### Cleaned columns: \n",data.columns)

### <a id='toc4_2_2_'></a>[Duplicates](#toc0_)

Duplicate rows are also redundancy.

In [ ]:
# Get duplicated rows
print("### Data Cleaning ### Check duplicate rows and count the number: \n",data_raw.duplicated().sum())

There are no duplicate rows in the dataset.

# <a id='toc5_'></a>[Descriptive Statistics](#toc0_)

Checking the data after cleansing unsing descriptive statistics (Mukhiya & Ahmed, 2020, p. 76).

In [ ]:
# Print some information about the DataFrame
print("### Descriptive Statistics ### DataFrame info: \n")
data.info()

Analyze the data using descriptive statistics (Mukhiya & Ahmed, 2020, p. 145-158):

In [ ]:
print("### Descriptive Statistics ### DataFrame describe: \n", data.describe())

Findings:
- every column contains 1152 elements, the data set therefore has no gaps
- n_sby has a standard deviation of zero, is thus constant
- n_sick varies between 36 and 119 
- the number of calls have a high standard deviation, i.e. fluctuate strongly
- the number of stubstitude drivers to be activated is on average approx. 35

# <a id='toc6_'></a>[Data Analysis](#toc0_)

This is the most important part to get insights from the data (Mukhiya & Ahmed, 2020, p. 81).

## <a id='toc6_1_'></a>[Correlation](#toc0_)

Finding correlated columns helps to identify the relation of the potential features (Mukhiya & Ahmed, 2020, p. 285).

In [ ]:
# Using seaborn for statistical data visualization
import seaborn as sns

# Finding correlated colums with heatmap
sns.heatmap(data.corr(), annot=True, fmt='.2f', linewidths=2)
# and pairplot()
sns.pairplot(data)


Findings:
- n_sick has a positive correlation with n_duty
- calls has a weak positive correlation with n_duty
- calls has a strong positive correlation with sby_need
- sby_need has a very strong positive correlation with dafted

It should be noted, that dropping the column drafted is the better choice than dropping sby_need, because sby_need contains more information! This can be checked by calculating it the other way around. Adding the value of n_sby to every value in dafted which is not zero is not totally equal to the column dafted, exactly at the points where dafted has values below the value of n_sby.

## <a id='toc6_2_'></a>[Visualizing Time Series](#toc0_)

Visualization enables further insights (Mukhiya & Ahmed, 2020, p. 224).

In [ ]:
# Create a figure with 6 subplots
fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6,1)
# Set figure width and height[inch]
fig.set_figwidth(16)
fig.set_figheight(30)

# Plot data: n_sick
ax1.plot(data.date, data.n_sick, '.')
# Set x-label
ax1.set_xlabel("date")
# Set y-label
ax1.set_ylabel("n_sick")
# Set title
ax1.set_title("Number of sick emergency drivers")

# Plot data: calls
ax2.plot(data.date, data.calls, '.')
# Set x-label
ax2.set_xlabel("date")
# Set y-label
ax2.set_ylabel("calls")
# Set title
ax2.set_title("Number of emergency calls")

# Plot data: n_duty
ax3.plot(data.date, data.n_duty, '.')
# Set x-label
ax3.set_xlabel("date")
# Set y-label
ax3.set_ylabel("n_duty")
# Set title
ax3.set_title("Number of emergency drivers on duty")

# Plot data: n_sby
ax4.plot(data.date, data.n_sby, '.')
# Set x-label
ax4.set_xlabel("date")
# Set y-label
ax4.set_ylabel("n_sby")
# Set title
ax4.set_title("Number of available substitude drivers")

# Plot data: sby_need
ax5.plot(data.date, data.sby_need, '.')
# Set x-label
ax5.set_xlabel("date")
# Set y-label
ax5.set_ylabel("sby_need")
# Set title
ax5.set_title("Number of substitude drivers to be activated")

# Plot data: dafted
ax6.plot(data.date, data.dafted, '.')
# Set x-label
ax6.set_xlabel("date")
# Set y-label
ax6.set_ylabel("dafted")
# Set title
ax6.set_title("Number of additional duty drivers")

Findings:
- In principle, a seasonal pattern and a slight upward trend can be discerned in the data
- The finding that the two columns sby_need and dafted are strongly correlated can be confirmed visually

## <a id='toc6_3_'></a>[Decomposition of variables](#toc0_)

A decomposition of variables into the major components will help during the modeling process (Atwan 2022, p. 299).

In [ ]:
# Using statsmodel for decomposition analysis
from statsmodels.tsa.seasonal import seasonal_decompose

# Set datetime as index so that the frequency of the data can be automatically detected
data = data.set_index(data.date)

# Decompose the data columns
# n_sick
# Create a figure with 4 subplots
fig, axes = plt.subplots(4,1)
# Set figure width and height[inch]
fig.set_figwidth(16)
fig.set_figheight(20)
plt.suptitle("n_sick")
res = seasonal_decompose(data.n_sick, model='additive', period=365) 
res.observed.plot(ax=axes[0])
axes[0].set_ylabel("observed")
res.trend.plot(ax=axes[1])
axes[1].set_ylabel("trend")
res.seasonal.plot(ax=axes[2])
axes[2].set_ylabel("seasonal")
res.resid.plot(ax=axes[3])
axes[3].set_ylabel("resid")
plt.show()
# calls
# Create a figure with 4 subplots
fig, axes = plt.subplots(4,1)
# Set figure width and height[inch]
fig.set_figwidth(16)
fig.set_figheight(20)
plt.suptitle("calls")
res = seasonal_decompose(data.calls, model='additive', period=365) 
res.observed.plot(ax=axes[0])
axes[0].set_ylabel("observed")
res.trend.plot(ax=axes[1])
axes[1].set_ylabel("trend")
res.seasonal.plot(ax=axes[2])
axes[2].set_ylabel("seasonal")
res.resid.plot(ax=axes[3])
axes[3].set_ylabel("resid")
plt.show()

There is an upward trend for both calls and n_sick, and a certain periodicity can be inferred from the decomposition as well.

## <a id='toc6_4_'></a>[Distributions](#toc0_)

Visual representation of the distributions can also help in analyzing the data (Atwan 2022, p. 258)

In [ ]:
# Create a figure with 6 subplots
#fig, (ax1, ax2, ax3, ax4, ax5, ax6) = plt.subplots(6,1)
# Set figure width and height[inch]
#fig.set_figwidth(16)
#fig.set_figheight(30)

# Create a figure with 6 subplots
fig, axes = plt.subplots(6, 1, figsize=(18, 30))
fig.suptitle('Histograms')

# n_sick
sns.histplot(ax=axes[0], data=data.n_sick)
# Set title
axes[0].set_title("Distribution of the number of sick emergency drivers")

# calls
sns.histplot(ax=axes[1], data=data.calls)
# Set title
axes[1].set_title("Distribution of the number of emergency calls")

# n_duty
sns.histplot(ax=axes[2], data=data.n_duty)
# Set title
axes[2].set_title("Distribution of the number of emergency drivers on duty")

# n_sby
sns.histplot(ax=axes[3], data=data.n_sby)
# Set title
axes[3].set_title("Distribution of the number of available substitude drivers")

# sby_need
sns.histplot(ax=axes[4], data=data.sby_need)
# Set title
axes[4].set_title("Distribution of the number of substitude drivers to be activated")

# dafted
sns.histplot(ax=axes[5], data=data.dafted)
# Set title
axes[5].set_title("Distribution of the number of additional duty drivers")

Based on the frequency distributions, a first analysis can be made regarding outliers:
- The distributions for n_sick and calls look inconspicuous, outliers cannot be assumed according to the present state of knowledge, since the range of values looks plausible
- The distributions to sby_need and dafted are extremly positive skewed. A statement about possible outliers is therefore not possible here.

## <a id='toc6_5_'></a>[Get the best-fitted distribution](#toc0_)

Identify the best-fitted distribution for specific data colums.

### <a id='toc6_5_1_'></a>[n_sick](#toc0_)

In [ ]:
# Library for fitting data to distributions
from fitter import Fitter

# Create a fitter instance
f = Fitter(data.n_sick,
#            distributions=['norm',
#                          'lognorm',
#                          "beta",
#                          "burr",
#                          "norm"]
)

# # Comment out because it is compute-intensive: Activate if new results are required
# Fit the data
#f.fit()
# Generate the fitted distribution summary
#f.summary()

Based on the sumsquare_error value the best distribution is the johnsonsb distribution (a Johnson SB continuous random variable).

### <a id='toc6_5_2_'></a>[calls](#toc0_)

In [ ]:
# Create a fitter instance
f = Fitter(data.calls,
#            distributions=['norm',
#                          'lognorm',
#                          "beta",
#                          "burr",
#                          "norm"]
)
           
# # Comment out because it is compute-intensive: Activate if new results are required
# Fit the data
#f.fit()
# Generate the fitted distribution summary
#f.summary()

Based on the sumsquare_error value the best distribution is the gennorm distribution (a generalized normal continuous random variable).

## <a id='toc6_6_'></a>[Boxplot](#toc0_)

Boxplots are a good choice to analyze potential outliers (Atwan 2022, p. 259).

In [ ]:
# Create a figure with 6 subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Boxplots')

# n_sick
sns.boxplot(ax=axes[0, 0], data=data.n_sick)
# Set y-label
axes[0, 0].set_ylabel("n_sick")
# Set title
axes[0, 0].set_title("Number of sick emergency drivers")

# calls
sns.boxplot(ax=axes[0, 1], data=data.calls)
# Set y-label
axes[0, 1].set_ylabel("calls")
# Set title
axes[0, 1].set_title("Number of emergency calls")

# n_duty
sns.boxplot(ax=axes[0, 2], data=data.n_duty)
# Set y-label
axes[0, 2].set_ylabel("n_duty")
# Set title
axes[0, 2].set_title("Number of emergency drivers on duty")

# n_sby
sns.boxplot(ax=axes[1, 0], data=data.n_sby)
# Set y-label
axes[1, 0].set_ylabel("n_sby")
# Set title
axes[1, 0].set_title("Number of available substitude drivers")

# sby_need
sns.boxplot(ax=axes[1, 1], data=data.sby_need)
# Set y-label
axes[1, 1].set_ylabel("sby_need")
# Set title
axes[1, 1].set_title("Number of substitude drivers to be activated")

# dafted
sns.boxplot(ax=axes[1, 2], data=data.dafted)
# Set y-label
axes[1, 2].set_ylabel("dafted")
# Set title
axes[1, 2].set_title("Number of additional duty drivers")

Based on the boxplots, the following potential outliers can be suspected:
- Only a few n_sick and calls values outside the whiskers, which could be defined as outliers
- n_duty and n_sby do not contain outliers
- Boxplots also confirm the positive skew in sby_need and dafted.

## <a id='toc6_7_'></a>[Skewness](#toc0_)

Skewness of the columns

In [ ]:
print("Skewness of the columns:  \n", data.skew(axis='index'))

The columns sby_need and dafted are highly skewed!

## <a id='toc6_8_'></a>[Stationarity](#toc0_)

For some time series forecasting techniques the data must be stationary (Atwan 2022, p. 308).

In [ ]:
# Using statsmodel for detecting stationarity
from statsmodels.tsa.stattools import adfuller, kpss

# Iterate over all columns which could be used as model input (so all except n_sby) except column one (date)
for column in data.iloc[:,1:].columns.drop("n_sby"):
    print("Data Strationarity of column", column)
    # Augmented Dickey-Fuller Test
    adf_output = adfuller(data[column])
    # Decision based on pval
    adf_pval = adf_output[1]
    if adf_pval < 0.05: 
        print("ADF Decision: Data is stationary")
    else:
        print("ADF Decision: Data is non-stationary")
    # Kwiatkowski-Phillips-Schmidt-Shin Test
    data[column].dropna()
    kpss_output = kpss(data[column])
    # Decision based on pval
    kpss_pval = kpss_output[1]
    if kpss_pval >= 0.05: 
        print("KPSS Decision: Data is stationary")
    else:
        print("KPSS Decision: Data is non-stationary")

There are some columns which contain non-stationary data and must be considered during the data preparation phase.

# <a id='toc7_'></a>[Data Quality](#toc0_)

Basically, it is multivariate time series data with good quality. Only the date column was converted for better processing and no missing values could be detected. In some cases, variables show strong correlations with each other what needs to be considered in the data preparation phase and the feature selection phase.
The data types of the variables match the specification and the value ranges are mostly plausible, but should be checked for outliers in individual cases. Data of some columns are highly skewed, which must be considered when detecting outliers.
In principle, a seasonal pattern and a slight upward trend can be discerned in the data.